In [ ]:
import numpy as np
%matplotlib notebook
from scipy import fftpack
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import SVG
from style import *

def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114]) 

# Teoría de la Información

## Contenidos

- Cantidad de información y entropía
- Codificación de fuente
- Codificación de canal


# Codificación de fuente

- Es el proceso que asigna un código a cada mensaje
- El largo de las "palabras de código" puede ser **fijo** o **variable**
- **Ejemplo:** Queremos codificar las letras del alfabeto (27) usando dos símbolos (código binario)
    - Para largo fijo necesitamos al menos 5 bits ($2^5 = 32$)
- Pero algunas letras se ocupan más que otras (*e.g.* vocales) 
- ¿Podemos aprovechar esto para comprimir un mensaje?

In [ ]:
import string
from collections import Counter

with open('quijote.txt', 'r') as file:
    texto = file.read()
print(texto)
texto = texto.translate({ord(k): None for k in string.digits})
for symbol in string.punctuation:
    texto = texto.replace(symbol, "")
texto = texto.replace(" ", "").replace("\n", "")
texto = texto.lower().encode('ascii', 'ignore').decode("utf-8")
print("\nLargo del texto: {0}, Cantidad de bits: {1}".format(len(texto), len(texto)*5))
Counter(texto).most_common()

**Intuición:** Podríamos reducir la cantidad de bits si usamos códigos más cortos para las letras más frecuentes

***

# Teoría de la información

- Estudio matemático sobre la cuantificación y transmisión de la información 
- Propuesto por **Claude Shannon** en 1948: *A Mathematical Theory of Communication*
- Proporciona medidas para describir la información de un proceso: **Entropía** e **Información Mutua**
- Aplicaciones en telecomunicaciones, computación y biología (genética)
- Fuerte influencia en la teoría de codificación y compresión

### Las dos fuentes

Sean dos fuentes **F1** y **F2** que pueden emitir uno entre cuatro símbolos: $A$, $B$, $C$ o $D$

**F1** es completamente aleatoria, es decir: $P(A) = P(B) = P(C) = P(D) = \frac{1}{4}$

Si queremos predecir el próximo valor emitido por **F1** ¿Cúal es el número mínimo de preguntas con respuesta si/no que debemos hacer?


<img src="images/information1.svg" width="600">

> La respuesta es 2 para cualquiera de los símbolos

**F2** en cambio emite $A$, $B$, $C$ y $D$ con probabilidades $P(A) =\frac{1}{2}$, $P(B) =\frac{1}{4}$, $P(C) = \frac{1}{8}$ y $P(D) =\frac{1}{8}$, respectivamente

Si queremos predecir el próximo valor retornado por **F2** ¿Cúal es el número mínimo de preguntas con respuesta si/no que debemos hacer? 

<img src="images/information2.svg" width="800">

> La respuesta es 1 para $A$, 2 para $B$ y 3 para $C$ y $D$



# Cantidad de información (según Shannon)

La cantidad de información de un símbolo $x$ es el logaritmo en base dos del recíproco de su probabilidad de aparición

$$
I(x) = \log_2 \left(\frac{1}{P(x)} \right) = \log_2 P(x)^{-1} = - \log_2 P(x),
$$

que es equivalente a la mínima cantidad de preguntas si/no que debemos hacer para adivinar su valor

La cantidad de información se mide en **bits**

>Un **bit** es la cantidad de información que se requiere para escoger entre **dos** alternativas equiprobables

La cantidad de información es también llamada **sorpresa**

> Mientras más improbable es un símbolo, más nos sorprendemos cuando observamos que ocurre

# Entropía

Sea una variable aleatoria $X$ (fuente) con $N$ resultados posibles (símbolos) $\{x_1, x_2, \ldots, x_N\}$

Cada símbolo $x_i$ tiene una probabilidad $p_i \in [0, 1]$ y $\sum_{i=1}^N p_i = 1$ 

Cada símbolo tiene una cantidad de información  $I(x_i) = -\log_2 \left( p_i \right)$ 

Definimos la **cantidad de información promedio** de $X$ como
$$
\begin{align}
H (X) &= \mathbb{E}_{x\sim X} \left [ - \log P(X=x) \right ]  \nonumber \\
&= - \sum_{i=1}^N P(X=x_i) \log_2 P(X=x_i)  \nonumber \\
&= - \sum_{i=1}^N p_i \log_2 p_i  \quad \text{[bits/símbolo]} \nonumber
\end{align}
$$
que se conoce como **Entropía de Shannon** 

### Propiedades
- La entropía es siempre positiva $H(X) \geq 0$. La igualdad se cumple si un $x_i$ tiene $p_i=1$ (caso más predecible)
- La entropia está acotada $H(X) \leq H_0$, donde $H_0= \log_2(N)$ es la entropia si $p_i = \frac{1}{N}~ \forall i$ (caso menos predecible)
- La redundancia de $X$ es $1 - H(X)/H_0$

> Mientras más predecible es $X$ menor es su entropía y mayor es su redundancia




### El retorno de las dos fuentes

En promedio, ¿Cuántas preguntas por símbolo hace la fuente **F1**?

> $1 \frac{1}{4} + 1 \frac{1}{4} + 1 \frac{1}{4} + 1 \frac{1}{4} = 2$ preguntas por símbolo. Su entropía es $2$ [bits]

En promedio, ¿Cuántas preguntas por símbolo hace la fuente **F2**?

> $1 \frac{1}{2} + 2 \frac{1}{4} + 3 \frac{1}{8} + 3 \frac{1}{8} = 1.75$ preguntas por símbolo. Su entropía es $1.75$ [bits]

Si cada fuente retorna un mensaje de 100 símbolos ¿Cúanta información produjo cada una?

> **F1** produce 200 bits mientras que **F2** produce 175 bits

Mientras más predecible menos información se necesita




### Ejemplo: Moneda con truco

- Sea una variable aleatoria $X$ que modela el resultado de lanzar una moneda
- Asumamos que el resultado puede tomar solo dos valores: Cara $o$ o Cruz $x$
- La probabilidad de que salga cara es $p_o = p$
- La probabilidad de que salga cruz es $p_x = 1- p$
- Luego la entropía es 

$$
\begin{align}
H(X) &= -\sum_{i=1}^2 p_i \log_2 p_i \nonumber \\ 
&= -p_x \log (p_x) - p_o \log p_o \nonumber \\
&= - p \log(p) - (1-p) \log(1-p)
\end{align}
$$

Reflexione:
- ¿En que casos la entropía es mínima? ¿En qué caso es máxima?
- ¿Puedes relacionar la entropía con la aleatoridad/sorpresa del resultado de lanzar la moneda?


Ojo: $\lim_{z\to 0^+} z \log 1/z = 0$

In [ ]:
p = np.linspace(0.01, 0.99, num=100)
H = -p*np.log2(p) - (1-p)*np.log2(1-p)
fig, ax = plt.subplots(1, figsize=(6, 4), sharex=True)
ax.set_xlabel('p')
ax.plot(p, -np.log2(p), label='I(o)', lw=3)
ax.plot(p, -np.log2(1-p), label='I(x)', lw=3)
ax.plot(p, H, label='H(X)', lw=3)
ax.set_ylim([0, 3])
plt.legend();

**Ejercicio:**

Sea una fuente que escupe un entero x que está entre 0 y 31

Considere el resultado de las siguientes preguntas ¿Cúal tiene mayor entropía?
- ¿Es x igual a 0?
- ¿Es x un número primo?
- ¿Es x mayor a 15?

¿Cuál es el número mínimo de preguntas con respuesta si/no que se deben hacer para adivinar el valor de x?


### Ejemplo: Meteorólogos del siglo XIX

- Nos encontramos a finales del siglo XIX
- La estación meteorológica de Niebla hace una predicción del tiempo en Valdivia
- Esta información se envía a Valdivia a través de telegrafo
- Calcule la cantidad de información promedio que envía la estación a Valdivia en cada escenario usando la **entropía de Shannon**

**Escenario 1:** Dos posibilidades: Lluvia y nublado, con probabilidad $1/2$ y $1/2$, respectivamente

**Escenario 2:** Una posibilidad: Lluvia, con probabilidad $1$

**Escenario 3:** Cuatro posibilidades: Lluvia, Nublado, Nubosidad parcial, soleado, con probabilidad $1/2$, $1/4$, $1/8$ y $1/8$, respectivamente

1. Las probabilidades de cada mensaje son $2^{-1}$, $2^{-2}$, $2^{-3}$ y $2^{-3}$
1. La cantidad de información de cada mensaje es: 1, 2, 3 y 3 bits, respectivamente
1. La entropía es $1/2 + 1/2 + 3/8 + 3/8 = 1.75$ bits

Para el **escenario 3** códifique las alternativas usando un alfabeto de códigos binarios

> ¿Cómo le asignamos un código a cada alternativa?

### Código de ancho fijo

- Tenemos cuatro estados, necesitamos 2 bits
- Cada estado: 00, 01, 10, 11
- En este caso resulta equivalente a asumir equiprobabilidad 
- La entropía es 2 bits

### Código de ancho variable (prefijo)

- Se usa 1, 2, 3 y 3 bits para cada estado, según su probabilidad de aparición
- La entropía es 1.75 bits
- Podemos describir este escenario según
    - Primera decisión equiprobable: Lluvia **(0)** vs El resto (1)
    - Segunda decisión equiprobable: Nublado **(10)** vs El resto menos lluvia (11)
    - Tercera decisión equiprobable: Nubosidad parcial **(110)** vs soleado **(111)**
- Podemos escribir esto como un dendograma

<center><img src="images/dendogram.png" width="600"></center>

- Algoritmo de codificación con forma de árbol en base 2
- Los mensajes codificados están en las hojas del árbol
- **Código préfijo**: Ningún código puede ser prefijo de otro. 
- El código prefijo garantiza decodificación sin ambiguedad


**Ejercicio**

Decodifique la predicción del tiempo para los próximos tres días: 101100 

**Ejemplo de código ambiguo** 

Si el código de lluvia fuera **1** en lugar de 0, decodifique el siguiente mensaje: 11111



### Ejemplo: Entropía y cantidad de bits del fragmento del famoso texto


In [ ]:
# Código de largo fijo:
print(5*len(texto))
# Código de largo variable:
freq = np.array(list(Counter(texto).values()))
p = freq/np.sum(freq)
print(int(-np.sum(p*np.log2(p))*len(texto)))

**Reflexionemos:** 
- ¿Es la codificación de largo variable *lossless* o *lossy*?
- En ciertos casos las palabras son más largas de lo que eran originalmente, ¿Cómo comprimimos entonces?

## Codificación de Huffman

Un algoritmo sencillo de codificación de tipo prefijo:

1. Se estima la probabilidad $p_i$ de cada símbolo
1. Se ordenan los símbolos en orden descendente según $p_i$
1. Juntar los dos con probabilidad menor en un grupo, su probabilidad se suma
1. Volver al paso 2 hasta que queden dos grupos
1. Asignarle 0 y 1 a las ramas izquierda y derecha del árbol, respectivamente
1. El código resultante se lee desde la raiz hasta la hoja

<center><a href="http://www.skylondaworks.com/sc_huff.htm"><img src="images/huff.gif" width="600"></a></center>


**Debilidad de Huffman:** 
- Códigos con diccionarios/probabilidades variables
- En ese caso combiene usar codificación aritmética o Lempel-Ziv



***
***
***

Hemos visto que en general las señales/datos tienen alta redundancia 

> Piense por ejemplo en el caso de las imágenes o el lenguaje (contexto)

Hemos visto también como comprimir datos

En particular 
1. Transformamos los datos tal de hacerlos "más independientes" y opcionalmente los cuantizamos
1. Codificamos los datos con una distribución que sea óptima para el canal de transmisión

Este último paso es lo que llamamos **codificación de fuente**

A continuación revisaremos un importante teorema enunciado por Shannon respecto a la **codificación de un mensaje**

# Teorema de codificación de fuente de Shannon (*Source coding theorem*)


>Dada una variable aleatoria $X$ con entropía $H(X)$ existe una codificación de largo variable cuyo largo de palabra promedio $\bar L$ satisface

>$$
H(X) \leq \bar L < H(X) + 1
$$

Es decir que el límite inferior teórico del largo de palabra es $H(X)$

Esta codificación sin pérdida y de largo variable la llamamos **codificación entrópica** 

Este teorema 
- nos dice cuanto podemos comprimir una señal sin que hayan pérdidas antes de enviarla por un canal (libre de ruido)
- justifica la definición de entropía como medida de la cantidad de información


Otra forma de ver el teorema

Sea una fuente $X$ que emite $N$ mensajes. 

> Los N mensajes pueden comprimirse en $N H(X)$ [bits] o más con riesgo de pérdida despreciable ($N\to\infty$)

Por el contrario

> Si comprimimos en menos de $N H(X)$ [bits] la pérdida está garantizada

#### Probando el teorema

Sea una codificación C para una variable aleatoria $X$ con N posibles símbolos

Cada símbolo $x_i$ tiene una probabilidad de ocurrencia $p_i \in [0, 1]$ con $\sum_i p_i = 1$ y un largo de código $L_i$

Luego el largo promedio de los códigos es

$$
\bar L = \sum_{i=1}^N p_i L_i
$$

¿Qué valores de $L_i$ resultan en el menor $\bar L$? 

> El largo óptimo es $L_i^* = -\log_2 p_i$ y el promedio sería $\bar L^* = H(X)$

Digamos que proponemos otro largo $\hat L_i = - \log_2 q_i$, asumiendo que $\sum_i q_i = 1$

Luego el largo promedio sería

$$
\begin{align}
\bar L &= \sum_{i=1}^N p_i \hat L_i  = - \sum_{i=1}^N p_i \log_2 q_i \nonumber \\
&= - \sum_{i=1}^N p_i \log_2 q_i - \sum_{i=1}^N p_i \log_2 p_i + \sum_{i=1}^N p_i \log_2 p_i \nonumber \\
&= - \sum_{i=1}^N p_i \log_2 p_i + \sum_{i=1}^N p_i \log_2 \frac{p_i}{q_i} \nonumber \\
&= H(X) + \sum_{i=1}^N p_i \log_2 \frac{p_i}{q_i} \geq H(X) \nonumber
\end{align}
$$
> Con esto probamos que no hay mejor largo que $-\log_2 p_i$

Notemos que los $L_i^*$ no tendrían porque ser un número enteros 

> En general la codificación óptima cumple: $H(X) \leq \bar L^* < H(X) + 1$

- Se puede estar entre esas cotas con el algoritmo de Huffman
- La codificación aritmética en cambio casi siempre llega a la cota inferior
- La codificación de Huffman y aritmética son **codificaciones entrópicas**

## Divergencia de Kullback-Leibler (KL)

**Divergencia:** Medida de similitud entre distribuciones estadísticas (No es exactamente una distancia)

Sean dos densidades $P=\{p_1, p_2, \ldots, p_N\}$ y $Q=\{q_1, q_2, \ldots, q_N\}$ sobre una misma variable $x$. Su **divergencia de KL** es

$$
\begin{align}
D_{\text{KL}} \left ( P||Q \right) &= \sum_{i=1}^N p_i \log_2 \frac{p_i}{q_i} \nonumber \\
&= -\sum_{i=1}^N p_i \log_2 q_i + \sum_{i=1}^N p_i \log_2 p_i \nonumber \\
&= H_Q(P) -H(P) \geq 0 \nonumber 
\end{align}
$$

donde 
- $H(P)$ es la entropía de $P$ 
- $H_Q(P)$ es la **entropía cruzada** de $P$ con $Q$
- La desigualdad final se conoce como [desigualdad de Gibbs](https://en.wikipedia.org/wiki/Gibbs%27_inequality)
- La igualdad se cumple sólo si $P=Q$

La divergencia de KL también se conoce como la **entropía relativa de P con respecto a Q**


> La divergencia de KL nos dice la "cantidad extra" de bits necesarios para codificar P usando un código optimizado para Q

> La divergencia de KL mide cantidad de información perdida cuando uso Q para aproximar P

# Información Mutua

La información mutua entre dos variables aleatorias $X$ e $Y$ se puede definir de varias maneras

$$
\begin{align}
\text{MI}(X, Y) &= H(X) + H(Y) - H(X, Y)  \nonumber \\
&= H(Y) - H(X|Y) = H(X) - H(Y|X)   \nonumber \\
&= D_\text{KL} \left ( P_{XY} || P_{X} P_{Y} \right) \nonumber \\
&= \sum_{i} \sum_{j} P_{XY}(X=x_i, Y=y_j) \log_2 \frac{P_{XY}(X=x_i,Y=y_j)}{P_{X}(X=x_i) P_Y(Y=y_j)}, \nonumber
\end{align}
$$

donde 
- $H(X,Y)$ es la entropía conjunta
    - Cantidad de información promedio en bits de $X$ e $Y$
- $H(X|Y)$ es la entropía de $X$ condicionada en $Y$
    - Cantidad de información promedio en bits de $X$ considerando que conocemos $Y$
    - Incerteza de $X$ dado que observamos $Y$
    - Recordemos la relación entre densidad conjunta y condicional: $p(x|y)p(y) = p(x,y)$
- Se cumple que $H(X)+H(Y) \geq H(X,Y)$ por lo tanto $\text{MI}(X,Y) \geq 0$
- Si $X$ e $Y$ son independientes entonces $H(X|Y)=H(X)$ y $H(X,Y) = H(X) + H(Y)$ y $\text{MI}(X,Y) = 0$

> La información mutua mide la información compartida por $X$ e $Y$, es decir que tan dependientes son entre sí

> La información mutua nos dice la información promedio en bits que ganamos sobre $Y$ dado que observamos $X$ (y viceverza)

> La información mutua nos dice la incerteza promedio de $Y$ que eliminamos al conocer $X$ 

# Canal con ruido

Hasta ahora hemos asumido que el canal está libre de ruido

El objetivo original de Shannon era

> "Comunicación robusta a través de un canal ruidoso"

El ruido disminuye la **capacidad** del canal

### Ejemplo: Un cisne navegando por  un canal ruidoso

Imagemos que queremos transmitir una imagen binaria $X$ por un canal con ruido

El canal le cambia el valor a un 10% de los píxeles

Llamamos $Y$ a la imagen que sale del canal

Este canal de comunicación se conoce como [canal binario simétrico](https://en.wikipedia.org/wiki/Binary_symmetric_channel)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3), tight_layout=True)
plt.rcParams['image.cmap'] = 'gray'

th_binary, p_noise = 0.5, 0.1
img_swan_gray = plt.imread('images/gray-swan.png')[:, :, 0]
img_swan = (img_swan_gray > th_binary).astype('uint8')
Npix = len(img_swan.ravel())
p = np.random.rand(img_swan.shape[0], img_swan.shape[1])
img_noisy_swan = img_swan.copy()
img_noisy_swan[p <= p_noise] = 1-img_noisy_swan[p <= p_noise]

ax[0].imshow(img_swan); ax[0].axis('off')
ax[1].imshow(img_noisy_swan); ax[1].axis('off');

¿Cual es la entropía de cada imagen?

In [ ]:
def entropy_binary_image(img):
    p = np.count_nonzero(img.ravel())/len(img.ravel())
    return -p*np.log2(p) - (1-p)*np.log2(1-p)

HX = entropy_binary_image(img_swan)
HY = entropy_binary_image(img_noisy_swan)
print("Entropía imagen limpia H(X): {0:0.4f} [bits/pixel]".format(HX))
print("Entropía imagen sucia H(Y): {0:0.4f} [bits/pixel]".format(HY))

¿Cual es la entropía conjunta?

In [ ]:
Npix = len(img_swan.ravel())
p00 = np.count_nonzero((img_swan == 0) & (img_noisy_swan == 0))/Npix
p01 = np.count_nonzero((img_swan == 0) & (img_noisy_swan == 1))/Npix
p10 = np.count_nonzero((img_swan == 1) & (img_noisy_swan == 0))/Npix
p11 = np.count_nonzero((img_swan == 1) & (img_noisy_swan == 1))/Npix
print(np.array([p00, p01, p10, p11]))

HXY = -(p00*np.log2(p00) + p01*np.log2(p01) + p10*np.log2(p10) + p11*np.log2(p11))
print("Entropía conjunta H(X,Y): {0:0.6f} [bits/pixel]".format(HXY))

¿Cuáles son las entropía condicionales?

In [ ]:
print("H(X|Y): {0:0.6f} [bits/pixel]".format(HXY-HY))
print("H(Y|X): {0:0.6f} [bits/pixel]".format(HXY-HX))

¿Cual es la información mutua?

In [ ]:
MIXY = HX + HY - HXY
print("Información mutua IM(X,Y): {0:0.6f} [bits/pixel]".format(MIXY))

¿Cúal es la entropía del ruido?

In [ ]:
H_noise = -p_noise*np.log2(p_noise) - (1-p_noise)*np.log2(1-p_noise)
print("Entropía del ruido H(N): {0:0.6f} [bits/pixel]".format(H_noise))

Consideremos que $Y = X + N$

Entonces
$$
H(Y|X) = H(X+N|X) = H(N|X) = H(N)
$$

> En un canal ruidoso la entropía condicional de la salida dada la entrada es equivalente a la entropía del ruido añadido a la entrada

<img src="images/entropy_mi_diagram.png">

La eficiencia de la transmisión está dada por la proporción de entropía de $Y$ que es compartida por $X$

$$
E = \frac{\text{MI}(X,Y)}{H(Y)} \in [0, 1]
$$



In [ ]:
MIXY/HY

Un 37% de la entropía de la salida depende de la entrada

¿Qué ocurre con la información mutua y con la eficiencia de transmisión cuando el canal se vuelve más o menos ruidoso?

# Corrección de errores debidos al ruido

Queremos ser capaces de recuperar X a partir de Y

Cuando el canal tiene ruido necesitamos robustecer el código de X

Esto se logra **agregando redundancia** a nuestro código

- Enviar el mensaje varias veces: **repetition code**
- Agregar al código uno o más **bits de paridad**


### Ejemplo: *repetition code*

Si queremos enviar 0110011 lo repetimos una cierta cantidad de veces

$X$ = 000 111 111 000 000 111 111 

$N$ = 001 000 000 000 000 110 000

$Y$ = 001 111 111 000 001 001 111

Si aplicamos decodificación por mayoría: 0 1 1 0 0 **0** 1

Reducimos la probabilidad de error por un factor 3

La tasa es $R = k/n = 3$ donde $k$ son los bits de información y $n$ los bits transmitidos

### Ejemplo: paridad

Sea una tira binaria de 16 bits $s=[0,1,0,1,0,0,0,0,1,1,1,0, 0,1,1,0]$ Se ordena como una matriz de 4x4

$$
\begin{array}{|c c c c|c}
  0 & 1 & 0 & 1 & 0\\
  0 & 0 & 0 & 0 & 0\\
  1 & 1 & 1 & 0 & 1\\
  0 & 1 & 1 & 0 & 0 \\ \hline
  1 & 1 & 0 & 1 & 
\end{array}
$$

Si el número de 1's de una fila o columna es par se agrega un 0, de lo contrario se agrega un 1

Finalmente se crea una nueva tira de largo 24

$s_p=[0,1,0,1,\textbf{0}, 0,0,0,0,\textbf{0}, 1,1,1,0, \textbf{1}, 0,1,1,0, \textbf{0}, \textbf{1}, \textbf{1}, \textbf{0}, \textbf{1}]$ 

- Al recibir este código se comprueba que las paridades estén bien
- Si no lo están podriamos pedir nuevamente la tira binaria
- Aumentamos el mensaje de 16 a 24 [bits], la tasa es $R=16/24=0.666$

**Ejercicio:** Si tengo un string de NxN de largo y quiero proteger con bits de paridad ¿Cúantos bits agrego?


# Teorema de codificación de canal de Shannon (*Channel coding theorem*)

Se define la capacidad de un canal con entrada $X$ y salida $Y$ como

$$
C = \max_{P(X)} \text{MI}(X,Y) ~\text{[bits/símbolo]} 
$$

La distribución $P^*(X)$ que maximiza la capacidad del canal es la distribución de entrada óptima para ese canal

Notemos que si el canal no tiene ruido entonces $Y=X$ y la capacidad está dada por la entropía de $X$

> El ruido disminuye la capacidad de un canal

Al respecto Shannon enunció el siguiente teorema

> Sea un canal con capacidad $C$ y una fuente $X$ que transmite a una taza de $R$

>Si $R \leq C$ entonces existe un un largo de codificación para $X$ que puede transmitirse con error arbitrariamente pequeño

> Para una una probabilidad de error de bit aceptable $p_e$, se puede alcanzar una taza de transmisión
$$
R(p_e) = \frac{1}{1 + p_e \log_2(p_e) + (1-p_e) \log_2(1-p_e)}
$$

> Para un cierto $p_e$ no es posible alcanzar una taza mayor a $R(p_e)$

### Ejemplo: Canal binario simétrico

¿Cuál es la distribución de entrada que maximiza la información mutua del canal?

In [ ]:
p_flip, tol = 0.1, 1e-2
np.random.seed(0)
seed_image = np.random.rand(img_swan_gray.shape[0], img_swan_gray.shape[1])
flip_mask = np.random.rand(img_swan_gray.shape[0], img_swan_gray.shape[1]) <= p_flip

binarization_threshold = np.linspace(0.01, 0.999, num=100)
MIXY = []
for th in binarization_threshold:
    img_X = (img_swan_gray > th).astype('uint8')
    img_Y = img_X.copy()
    img_Y[flip_mask] = 1-img_Y[flip_mask] 
    p00 = np.count_nonzero((img_X == 0) & (img_Y == 0))/Npix
    p01 = np.count_nonzero((img_X == 0) & (img_Y == 1))/Npix
    p10 = np.count_nonzero((img_X == 1) & (img_Y == 0))/Npix
    p11 = np.count_nonzero((img_X == 1) & (img_Y == 1))/Npix
    HX = -(p00 + p01)*np.log2(p00 + p01+tol) -(p10 + p11)*np.log2(p10 + p11+tol)
    HY = -(p00 + p10)*np.log2(p00 + p10+tol) -(p01 + p11)*np.log2(p01 + p11+tol)
    HXY= -(p00*np.log2(p00+tol) + p01*np.log2(p01+tol) + p10*np.log2(p10+tol) + p11*np.log2(p11+tol))
    MIXY.append(HX + HY - HXY)
    
fig, ax = plt.subplots(2, 2, figsize=(9, 7), tight_layout=True)
ax[1, 0].axis('off'); ax[0, 1].axis('off'); ax[1, 1].set_xticks([0, 1]); 
ax[1,1].set_xlabel('X'); ax[1, 1].set_ylabel('P(X)');
ax[0, 0].set_ylabel('Información mutua')
ax[0, 0].set_xlabel('Umbral de binarización');
best_th = binarization_threshold[np.argmax(MIXY)]
ax[0, 1].text(0.0,0.5, "Mejor umbral: {0:0.4f}\nMejor MI(X,Y): {1:0.4f}".format(best_th, np.amax(MIXY)))
ax[0, 0].plot(binarization_threshold, MIXY); 
ax[1, 0].imshow((img_swan_gray > best_th).astype('uint8')); 
ax[1, 1].hist((img_swan_gray > best_th).astype('uint8').ravel(), 
              align='mid', range=[-0.5, 1.5],
              bins=2, density=True, histtype='bar', ec='black');

Digamos que la entrada $X$ tiene probabilidad $p$ de ser 0 y $(1-p)$ de ser 1 (imagen binaria) 

Es decir las *probabilidades a priori* son $P(X=0)=p$, $P(X=1)=(1-p)$ 

Dijimos que el canal cambia un 10\% de los pixeles de $X$

Entonces las verosimilitudes son $P(Y=1|X=1) = 0.9$, $P(Y=1|X=0) = 0.1$, $P(Y=0|X=1) = 0.1$, $P(Y=0|X=0) = 0.9$

Las probabilidades marginales de la salida son:
- $P(Y=1) = \sum_x P(Y=1|X=x)P(X=x)  = 0.9(1-p) + 0.1p = 0.9 - 0.8p $
- $P(Y=0) = \sum_x P(Y=0|X=x)P(X=x)  = 0.1(1-p) + 0.9p = 0.1 + 0.8p= 1 - P(Y=1) $ 

y su entropía es $H(Y) = - (0.9 - 0.8p) \log_2(0.9 - 0.8p) - (0.1 + 0.8p) \log_2(0.1 + 0.8p)$


La entropía condicional es 
$$
H(Y|X) = \sum_x H(Y|X=x)P(X=x) = - 0.9 \log_2(0.9) - 0.1 \log_2 (0.1)
$$
que no depende de $p$

El máximo de la información mutua:
$$
\frac{d}{dp} \text{MI}(X,Y) = \frac{d}{dp} H(Y) - \frac{d}{dp} H(Y|X) = \frac{d}{dp} H(Y) = 0 \rightarrow p^*=\frac{1}{2}
$$

Notemos que no importa que porcentaje de corrupción tenga el canal.

Finalmente la capacidad del canal es

$$
C = \text{MI}_p^* (X,Y) = 1 + 0.9 \log_2(0.9) + 0.1 \log_2 (0.1) = 0.531 \text{[bits]}
$$

> Si usamos P^* podemos transmitir sin perdidas a una taza $R=k/n=0.531$, es decir que por $k$ bits de información tenemos que transmitir aproximadamente $2k$ bits



## Teorema de Shannon-Hartley

- Sea un canal con ancho de banda B [Hz] (rango de frecuencias que un canal puede transmitir) y potencia de señal S [W] y potencia del ruido N [W] (aditivo blanco gaussiano), entonces su capacidad es

$$
C = B \log_2 \left(1 + \frac{S}{N} \right) \text{[bits/s]}
$$

- Si la velocidad de transmisión de un canal es R [bits/s] y R < C entonces la probabilidad de errores de comunicación tiende a cero.
- Las limitaciones de un sistema de comunicación son **Ancho de banda** y el ruido **Ruido**



http://home.iitk.ac.in/~adrish/Shannon/information_theory.php